In [1]:
# fix for windows memory leak with MKL
import os
import platform

if platform.system() == "Windows":
    os.environ["OMP_NUM_THREADS"] = "2"

# import libraries
import time
import random
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  # this is used for the plot of the graph
from scipy.stats import zscore, median_abs_deviation, skew, kurtosis
import joblib
# Sklearn classes
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    KFold,
)
from sklearn import metrics
from sklearn.metrics import make_scorer, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    LabelEncoder,
    MinMaxScaler,
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn import tree
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_blobs
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from helper import (
    draw_confusion_matrix,
    heatmap,
    make_meshgrid,
    plot_contours,
    draw_contour,
)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Sets random seed for reproducibility
SEED = 42
random.seed(SEED)

In [2]:
def preprocess_data(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Optimized function to clean the input DataFrame by dropping irrelevant columns 
    and handling outliers using vectorized operations.
    """
    # Identify numeric columns, excluding 'site'. This part is already efficient.
    numeric_cols = df.columns.drop('site')
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # --- Vectorized Outlier Removal ---
    # 1. Calculate group-wise mean and std for all columns at once.
    #    .transform() broadcasts the result back to the original DataFrame's shape.
    means = df.groupby('site')[numeric_cols].transform('mean')
    stds = df.groupby('site')[numeric_cols].transform('std')

    # 2. Calculate Z-scores for all data points in a single, vectorized operation.
    #    We'll fill stds of 0 with NaN to avoid division-by-zero errors, then fill
    #    resulting Z-score NaNs with 0, as a point with no deviation is 0 Z-score.
    z_scores = ((df[numeric_cols] - means) / stds.replace(0, np.nan)).fillna(0)
    
    # 3. Create a boolean mask to identify outliers.
    #    This mask is True for any value where the absolute Z-score > 5.
    mask = z_scores.abs() > 5

    # 4. Use the mask to replace outliers with NaN.
    #    df.where() keeps original values where the mask is False.
    df[numeric_cols] = df[numeric_cols].where(~mask, np.nan)
    # ------------------------------------

    # Calculate final median and standard deviation after handling outliers.
    # These are already efficient groupby aggregations.
    med = df.groupby('site').median().T
    std = df.groupby('site').std().T
    
    return df, med, std


In [3]:
# Load Data
data = pd.read_csv("Data_Raw.csv", header=1)
sites = data['site']
niki = pd.read_csv("Data_Niki.csv")
niki = niki.drop(index=[0,1])
niki['Engineer classification.1'] = (niki['Engineer classification.1'] == 'Release').astype(int)
y = niki['Engineer classification.1']

# Dropping Unnecessary Rows and Columns
filtered_ordered_columns = [col for col in niki['Parameter name'] if col in data.columns]
data = data[filtered_ordered_columns]
data = pd.concat([sites, data], axis=1)
data, med, std = preprocess_data(data)

# med = med.fillna(0)
# std = std.fillna(0)

C:\Users\nxg15046\AppData\Local\Temp\2\ipykernel_25036\1887821859.py:4: DtypeWarning: Columns (1,2,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255

In [4]:
# Standardizing by Row
z_med = med.apply(zscore, axis=1, nan_policy='omit')
z_med = pd.DataFrame(z_med.tolist(), index=med.index, columns=med.columns)
z_std = std.apply(zscore, axis=1, nan_policy='omit')
z_std = pd.DataFrame(z_std.tolist(), index=std.index, columns=std.columns)

In [5]:
# New Ground Truth Generation
# Define a threshold for significant deviation
threshold = 5

# Identify rows (test parameters) where any site has |z| > threshold in either z_med or z_std
anomalous_rows = (z_med.abs().gt(threshold).any(axis=1)) | (z_std.abs().gt(threshold).any(axis=1))

# Create the new ground truth: 0 for anomalous, 1 for normal
new_y = (~anomalous_rows).astype(int)
y.index = new_y.index
# new_y[y==0] = 0 (prevents Niki investigate becoming release)

changed_0_to_1 = ((y == 0) & (new_y == 1)).sum()
changed_1_to_0 = ((y == 1) & (new_y == 0)).sum()
stayed_0 = ((y == 0) & (new_y == 0)).sum()
stayed_1 = ((y == 1) & (new_y == 1)).sum()

# Print the results
print(f"Changed from 'investigate' to 'release': {changed_0_to_1}")
print(f"Changed from 'release' to 'investigate': {changed_1_to_0}")
print(f"Stayed as 'investigate': {stayed_0}")
print(f"Stayed as 'release': {stayed_1}")

print((y != new_y).sum())

Changed from 'investigate' to 'release': 18
Changed from 'release' to 'investigate': 1065
Stayed as 'investigate': 389
Stayed as 'release': 1199
1083


In [6]:
# Range (need to remove lower and upper limit)
z_med_min = z_med.min(axis=1)
z_std_min = z_std.min(axis=1)
z_med_max = z_med.max(axis=1)
z_std_max = z_std.max(axis=1)
z_med_range = z_med_max - z_med_min
z_std_range = z_std_max - z_std_min

## Inter-Quartile Range
Q1_med = z_med.quantile(0.25, axis=1)
Q3_med = z_med.quantile(0.75, axis=1)
z_med_iqr = Q3_med - Q1_med
Q1_std = z_std.quantile(0.25, axis=1)
Q3_std = z_med.quantile(0.75, axis=1)
z_std_iqr = Q3_std - Q1_std

## Median Absolute Deviation
z_med_mad = z_med.apply(median_abs_deviation, axis=1)
z_std_mad = z_std.apply(median_abs_deviation, axis=1)

## Skewness
med_skewness = med.skew(axis=1)
std_skewness = std.skew(axis=1)

## Kurtosis (Propensity for Outliers)
med_kurtosis = med.kurt(axis=1)
std_kurtosis = std.kurt(axis=1)

## Coefficient of Variation
eps = 1e-6
coeff_var = std / (med.abs() + eps) # avoid division by 0
mean_coeff_var = coeff_var.mean(axis=1)

## Experimental Features
# skew_min = skewness.min(axis=1)
# skew_max = skewness.max(axis=1)
# kurt_min = kurt.min(axis=1)
# kurt_max = kurt.max(axis=1)
# skew_range = skew_max - skew_min
# kurt_range = kurt_max - kurt_min

In [7]:
# Concatenating All Features
new_y.index = z_med.index
x = pd.concat([z_med_range, z_std_range, z_med_iqr, z_std_iqr, z_med_mad, z_std_mad, med_skewness, std_skewness, med_kurtosis, std_kurtosis, new_y], axis=1) # y or new_y for different ground truth

# Assigning New Column Names to Bypass Column Name Repeating Issue
num_columns = x.shape[1]
x.columns = x.columns.astype(str)
new_columns = ["z_med_range", "z_std_range", "z_med_iqr", "z_std_iqr", "z_med_mad", "z_std_mad", "med_skewness", "std_skewness", "med_kurtosis", "std_kurtosis", "y"]
x.columns = new_columns

corr_matrix = x.corr(numeric_only=True)
print(corr_matrix["y"].sort_values(ascending=False))
x = x.drop('y', axis=1)

y               1.000000
z_std_mad       0.768224
z_med_mad       0.641895
z_std_iqr       0.526109
z_med_iqr       0.411349
med_skewness    0.187971
med_kurtosis   -0.360014
z_med_range    -0.541792
z_std_range    -0.559828
std_kurtosis   -0.584545
std_skewness   -0.702624
Name: y, dtype: float64


In [8]:
# Train Test Split
train_raw, test_raw, target, target_test = train_test_split(x, new_y, test_size=0.3, stratify=new_y, random_state=0) # change to new_y

# Applying Pipeline to Train and Test Data
categorical_features = x.select_dtypes(exclude=["number"]).columns.tolist()
numerical_features = x.select_dtypes(include=["number"]).columns.tolist()
num_pipeline = Pipeline([("normalizer", StandardScaler())])
full_pipeline = ColumnTransformer([("num", "passthrough", numerical_features), ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)]) # changed from num_pipeline to passthrough

train = full_pipeline.fit_transform(train_raw)
test = full_pipeline.transform(test_raw)

joblib.dump(full_pipeline, 'pkl files/pipeline_noimpute.joblib')

['pkl files/pipeline_noimpute.joblib']

In [9]:
# Decision Tree Classifier

kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
f1_scorer = make_scorer(f1_score, pos_label=0)

clf_parameters = [{
    "max_depth": [2,4,8,16,32,64], 
    "min_samples_split": [2,4,8,16], 
    "criterion": ['gini', 'entropy']}]

clf = DecisionTreeClassifier(class_weight='balanced', random_state=42)
grid = GridSearchCV(clf, clf_parameters, cv = kf, scoring=f1_scorer)
grid.fit(train,target)
clf_res = pd.DataFrame(grid.cv_results_)
clf_predicted = grid.predict(test)
best_params_dt = grid.best_params_
relevant_columns = ['rank_test_score', 'mean_test_score', 'std_test_score']

top_3_models = clf_res[relevant_columns].sort_values(by='rank_test_score').head(3)

In [10]:
# XGBoost

xgb_params = {
    "n_estimators": [100, 200],
    "max_depth": [4, 6, 10],
    "learning_rate": [0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "scale_pos_weight": [5, 6],
    "gamma": [0, 1, 5]
}

xgb = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_grid = GridSearchCV(xgb, xgb_params, cv=kf, scoring=f1_scorer, n_jobs=-1, verbose=1)
xgb_grid.fit(train, target)
xgb_res = pd.DataFrame(xgb_grid.cv_results_)
best_params_xgb = xgb_grid.best_params_
relevant_columns = ['rank_test_score', 'mean_test_score', 'std_test_score']

top_3_xgb = xgb_res[relevant_columns].sort_values(by='rank_test_score').head(3)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


In [11]:
# LightGBM

lgbm_params = {
    "n_estimators": [300],
    "max_depth": [-1,],
    "learning_rate": [0.05, 0.1],
    "num_leaves": [31, 50, 100],
    "min_child_samples": [10, 20, 30],
    "colsample_bytree": [0.8, 1.0],
    "scale_pos_weight": [5, 6],
}

lgbm = LGBMClassifier(random_state=42)
lgbm_grid = GridSearchCV(lgbm, lgbm_params, cv=kf, scoring=f1_scorer, n_jobs=-1, verbose=1)
lgbm_grid.fit(train, target)
lgbm_res = pd.DataFrame(lgbm_grid.cv_results_)
best_params_lgbm = lgbm_grid.best_params_
relevant_columns = ['rank_test_score', 'mean_test_score', 'std_test_score']

top_3_lgbm = lgbm_res[relevant_columns].sort_values(by='rank_test_score').head(3)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
[LightGBM] [Info] Number of positive: 852, number of negative: 1017
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003899 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1963
[LightGBM] [Info] Number of data points in the train set: 1869, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.455859 -> initscore=-0.177026
[LightGBM] [Info] Start training from score -0.177026
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [12]:
# Random Forest

rf_params = {
    "n_estimators": [200, 300],
    "max_depth": [None, 16, 32],
    "min_samples_split": [2, 4, 8],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['sqrt', 'log2'],
    "criterion": ['gini', 'entropy']
}

rf = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_grid = GridSearchCV(rf, rf_params, cv=kf, scoring=f1_scorer, n_jobs=-1, verbose=1)
rf_grid.fit(train, target)
rf_res = pd.DataFrame(rf_grid.cv_results_)
best_params_rf = rf_grid.best_params_
relevant_columns = ['rank_test_score', 'mean_test_score', 'std_test_score']

top_3_rf = rf_res[relevant_columns].sort_values(by='rank_test_score').head(3)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


In [13]:
# Best Decision Tree

# Measure fit time
start_fit = time.time()
dt = DecisionTreeClassifier(**best_params_dt, random_state=42)
dt.fit(train, target)
end_fit = time.time()
joblib.dump(dt, 'pkl files/dt_raw_noimpute.joblib')
fit_time = end_fit - start_fit

# Measure prediction time
start_pred = time.time()
predicted = dt.predict(test)
end_pred = time.time()
dt_results = predicted
pred_time = end_pred - start_pred

# Metrics
accuracy = metrics.accuracy_score(target_test, predicted)
precision = precision_score(target_test, predicted, pos_label=0)
recall = recall_score(target_test, predicted, pos_label=0)
f1 = f1_score(target_test, predicted, pos_label=0)

# Print Results
print("%-12s %f seconds" % ('Fit Time:', fit_time))
print("%-12s %f seconds" % ('Prediction Time:', pred_time))
print("%-12s %f" % ('Test Accuracy:', accuracy))
print("%-12s %f" % ('Precision:', precision))
print("%-12s %f" % ('Recall:', recall))
print("%-12s %f" % ('F1 Score:', f1))

dt_metrics = ['Decision Tree', accuracy, precision, recall, f1, fit_time, pred_time]

Fit Time:    0.007704 seconds
Prediction Time: 0.000727 seconds
Test Accuracy: 0.990025
Precision:   0.993103
Recall:      0.988558
F1 Score:    0.990826


In [14]:
# Best XGBoost

# Measure fit time
start_fit = time.time()
xgb = XGBClassifier(**best_params_xgb, random_state=42)
xgb.fit(train, target)
end_fit = time.time()
joblib.dump(xgb, 'pkl files/xgb_raw_noimpute.joblib')
fit_time = end_fit - start_fit

# Measure prediction time
start_pred = time.time()
predicted = xgb.predict(test)
end_pred = time.time()
xgb_results = predicted
pred_time = end_pred - start_pred

# Metrics
accuracy = metrics.accuracy_score(target_test, predicted)
precision = precision_score(target_test, predicted, pos_label=0)
recall = recall_score(target_test, predicted, pos_label=0)
f1 = f1_score(target_test, predicted, pos_label=0)

# Print Results
print("%-12s %f seconds" % ('Fit Time:', fit_time))
print("%-12s %f seconds" % ('Prediction Time:', pred_time))
print("%-12s %f" % ('Test Accuracy:', accuracy))
print("%-12s %f" % ('Precision:', precision))
print("%-12s %f" % ('Recall:', recall))
print("%-12s %f" % ('F1 Score:', f1))

xgb_metrics = ['XGBoost', accuracy, precision, recall, f1, fit_time, pred_time]

Fit Time:    0.104879 seconds
Prediction Time: 0.004453 seconds
Test Accuracy: 0.988778
Precision:   0.993088
Recall:      0.986270
F1 Score:    0.989667


In [15]:
# Best LightGBM

# Measure fit time
start_fit = time.time()
lgbm = LGBMClassifier(**best_params_lgbm, random_state=42)
lgbm.fit(train, target)
end_fit = time.time()
joblib.dump(lgbm, 'pkl files/lgbm_raw_noimpute.joblib')
fit_time = end_fit - start_fit

# Measure prediction time
start_pred = time.time()
predicted = lgbm.predict(test)
end_pred = time.time()
lgbm_results = predicted
pred_time = end_pred - start_pred

# Metrics
accuracy = metrics.accuracy_score(target_test, predicted)
precision = precision_score(target_test, predicted, pos_label=0)
recall = recall_score(target_test, predicted, pos_label=0)
f1 = f1_score(target_test, predicted, pos_label=0)

# Print Results
print("%-12s %f seconds" % ('Fit Time:', fit_time))
print("%-12s %f seconds" % ('Prediction Time:', pred_time))
print("%-12s %f" % ('Test Accuracy:', accuracy))
print("%-12s %f" % ('Precision:', precision))
print("%-12s %f" % ('Recall:', recall))
print("%-12s %f" % ('F1 Score:', f1))

lgbm_metrics = ['LightGBM', accuracy, precision, recall, f1, fit_time, pred_time]

[LightGBM] [Info] Number of positive: 852, number of negative: 1017
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000318 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1963
[LightGBM] [Info] Number of data points in the train set: 1869, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.455859 -> initscore=-0.177026
[LightGBM] [Info] Start training from score -0.177026
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

c:\Users\nxg15046\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [16]:
# Best Random Forest

# Measure fit time
start_fit = time.time()
rf = RandomForestClassifier(**best_params_rf, random_state=42)
rf.fit(train, target)
end_fit = time.time()
fit_time = end_fit - start_fit
joblib.dump(rf, 'pkl files/rf_raw_noimpute.joblib')

# Measure prediction time
start_pred = time.time()
predicted = rf.predict(test)
rf_results = predicted
end_pred = time.time()
pred_time = end_pred - start_pred

# Metrics
accuracy = metrics.accuracy_score(target_test, predicted)
precision = precision_score(target_test, predicted, pos_label=0)
recall = recall_score(target_test, predicted, pos_label=0)
f1 = f1_score(target_test, predicted, pos_label=0)

# Print Results
print("%-12s %f seconds" % ('Fit Time:', fit_time))
print("%-12s %f seconds" % ('Prediction Time:', pred_time))
print("%-12s %f" % ('Test Accuracy:', accuracy))
print("%-12s %f" % ('Precision:', precision))
print("%-12s %f" % ('Recall:', recall))
print("%-12s %f" % ('F1 Score:', f1))

rf_metrics = ['Random Forest', accuracy, precision, recall, f1, fit_time, pred_time]

Fit Time:    0.593859 seconds
Prediction Time: 0.024124 seconds
Test Accuracy: 0.991272
Precision:   0.993119
Recall:      0.990847
F1 Score:    0.991982


In [17]:
# Summary Table of All Metrics

all_metrics = [dt_metrics, rf_metrics, xgb_metrics, lgbm_metrics]

# Create a DataFrame
metrics_df = pd.DataFrame(all_metrics, columns=[
    'Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Fit Time (s)', 'Prediction Time (s)'
])

# Print the table
metrics_df = metrics_df.sort_values(by='Accuracy', ascending=False)
print(metrics_df.to_string(index=False))


        Model  Accuracy  Precision   Recall  F1 Score  Fit Time (s)  Prediction Time (s)
Random Forest  0.991272   0.993119 0.990847  0.991982      0.593859             0.024124
Decision Tree  0.990025   0.993103 0.988558  0.990826      0.007704             0.000727
      XGBoost  0.988778   0.993088 0.986270  0.989667      0.104879             0.004453
     LightGBM  0.988778   0.993088 0.986270  0.989667      0.330539             0.010311
